In [26]:
%pylab
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import pandas as pd
from sqlalchemy import create_engine 
from __future__ import division


engine = create_engine('postgresql://postgres@localhost:5432/postgres')

instrument = 'EUR_USD'
fromDate = '2017-05-18 00:00:00+00'
toDate = '2017-05-20 00:00:00+00'
period = '5min'
fromtbl = 'oanda_%s_m1' % (instrument)

win_agg_group_sql = '''
  with intervals as (
  select start, start + interval '%s' as end
  from generate_series('%s', '%s', interval '%s') as start
  )
  select distinct
  intervals.start as dt,
  min(ask_l) over w as low,
  max(ask_h) over w as high,
  first_value(ask_o) over w as open,
  last_value(ask_c) over w as close,
  sum(v) over w as volume
  from
  intervals
  join %s tmb on
  tmb.dt >= intervals.start and
  tmb.dt < intervals.end
  window w as (partition by intervals.start)
  order by intervals.start;
''' % (period,fromDate,toDate,period,fromtbl)

orig_df = pd.read_sql_query(win_agg_group_sql, 
                       con=engine)
orig_df.dt = pd.to_datetime(orig_df.dt,utc=True).dt.tz_convert('Europe/Moscow')
orig_df.set_index(['dt'])
orig_df['dt']=orig_df.dt.dt.strftime('%m/%d/%Y %H:%M:%S')

orig_df['13m']=np.round(pd.Series(orig_df['close']).rolling(window=13).mean(),decimals=5)
orig_df['55m']=np.round(pd.Series(orig_df['close']).rolling(window=55).mean(),decimals=5)

wov_df=orig_df.drop(['volume'],axis=1)

#标准差
wov_df['std_5']=wov_df.std(axis=1) * 10**5

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# 均线系统选取

２根均线　１３/55 

## 取出标准差的最大值最小值


* ０－１８０的范围就是根据这个两个极值稍微两边拓展一点出来的．

## 画出 STD_5 的分布得出均值


## 连续性考察

图片上可以分析出低于５０的，持续粘连．




In [22]:
std_5_max = wov_df['std_5'].max()
std_5_min = wov_df['std_5'].min()

print 'max:[%f],min:[%f]' % (std_5_max,std_5_min)

max:[170.014999],min:[4.966555]


In [23]:
def get_stats(group):
    return {'count': group.count()}

#hist 简版直方图，没办法做范围约定
#wov_df['std_5'].hist(bins=10,alpha = 0.6)
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100 ,110,120,130,140,150,160,170, 180]
g_std = pd.cut(wov_df['std_5'], bins)
grouped = wov_df['std_5'].groupby(g_std)
bin_counts = grouped.apply(get_stats).unstack()
bin_counts.index =  [ '0~10','10~20','20~30','30~40','40~50', '50~60', '60~70', '70~80', '80~90', '90~100', '100~110', '110~120', '120~130', '130~140', '140~150', '150~160', '160~170', '170~180' ]
bin_counts.index.name = 'std range'
bin_counts.plot(kind='bar', alpha=0.5, rot=0)

In [24]:
print '参考均值最大范围［%f］' % bin_counts['count'].mean()

参考均值最大范围［30.000000］


In [25]:
# Two subplots, the axes array is 1-d
# 多图，共享时间轴
wov_df.dt

f, axarr = plt.subplots(2, sharex=True)
axarr[0].plot(pd.to_datetime(wov_df.dt),wov_df.close )
axarr[0].set_title('Sharing X axis')
axarr[1].plot(pd.to_datetime(wov_df.dt), wov_df.std_5)